In [33]:
import numpy as np
from qiskit.quantum_info import Statevector
from qiskit.circuit import ParameterVector
from qiskit.chemistry.components.variational_forms import UCCSD

x = ParameterVector('x', 3)
uvcc_varform = UCCSD(4, 2, qubit_mapping = 'jordan_wigner', two_qubit_reduction=False)
circuit = QuantumCircuit(4)
circuit.x(0)  # initial state
circuit.compose(uvcc_varform.construct_circuit(x), inplace=True)  # add UCC ansatz
circuit.draw()

┌───┐┌────────────────────┐┌────────────────────┐┌────────────────────┐
q_0: ┤ X ├┤0                   ├┤0                   ├┤0                   ├
     └───┘│                    ││                    ││                    │
q_1: ─────┤1                   ├┤1                   ├┤1                   ├
          │  Evolution^1(x[0]) ││  Evolution^1(x[1]) ││  Evolution^1(x[2]) │
q_2: ─────┤2                   ├┤2                   ├┤2                   ├
          │                    ││                    ││                    │
q_3: ─────┤3                   ├┤3                   ├┤3                   ├
          └────────────────────┘└────────────────────┘└────────────────────┘

In [42]:
# param shift rule
finite_difference = True 
shift = 0.1
gradient_circuits = {}
point = {x_i: np.random.random(1)[0] for x_i in x}
for x_i in x:
    # shifted parameters
    plus = circuit.bind_parameters({x_j: point_j + shift * int(x_i == x_j) for x_j, point_j in point.items()})
    minus = circuit.bind_parameters({x_j: point_j - shift * int(x_i == x_j) for x_j, point_j in point.items()})
    gradient_circuits[x_i] = (plus, minus)

In [44]:
for x_i, (plus, minus) in gradient_circuits.items():
    # print(plus.draw())
    # print(minus.draw())
    difference = Statevector.from_instruction(plus) - Statevector.from_instruction(minus)
    if finite_difference:
        difference /= 2 * shift
    print(x_i, np.round(difference.data, 2))

x[0] [-0.  +0.j -0.02+0.j  1.  -0.j -0.  +0.j  0.  -0.j -0.  +0.j -0.  +0.j
 -0.  -0.j -0.  +0.j -0.  +0.j -0.  +0.j  0.  -0.j  0.  -0.j -0.  -0.j
  0.  +0.j -0.  -0.j]
x[1] [ 0.-0.j -0.-0.j  0.-0.j -0.+0.j  0.+0.j  0.-0.j  0.-0.j  0.+0.j -0.+0.j
 -0.+0.j -0.-0.j -0.+0.j -0.-0.j -0.+0.j  0.+0.j  0.+0.j]
x[2] [-0.-0.j -0.+0.j -0.+0.j -0.+0.j -0.-0.j -0.-0.j  0.+0.j  0.+0.j -0.-0.j
  0.+0.j -0.-0.j -0.+0.j  0.+0.j  0.-0.j  0.+0.j -0.+0.j]
